In [1]:
import pandas as pd
import numpy as np
from glob import glob
import math

In [2]:
profiles = pd.read_csv('../bot_detection/profiles_with_bot_or_not.csv')
profiles = profiles.drop('followers_count', axis=1)

In [3]:
def update_predictions(df, profiles, user_column):            
    cols = list(df.columns)
    cols.remove('predicted')
    cols.append('predicted_x')
    cols.append('predicted_y')

    df = df.merge(profiles,left_on=user_column, right_on='username')[cols]
        
    print(sum(df['predicted_x'] == df['predicted_y'])/len(df))
    
    df = df.drop('predicted_x', axis=1)
    df = df.rename(columns={'predicted_y': 'predicted'})
    return df

In [6]:
# for file in glob('data/*'):
#     cascade = pd.read_csv(file)
#     cascade = update_predictions(cascade, profiles, 'User')
#     cascade.to_csv(file, index=None)

files = ["cascade_with_influence.csv"]

# for file in files:
#     df = pd.read_csv(file)
#     df = update_predictions(df, profiles, 'user_id')
#     df.to_csv(file, index=None)

In [7]:
df = pd.read_csv(files[0])
colname = 'cascade'

In [8]:
def get_percentage_of_post(df, percentage):
    return df[:math.ceil(df.shape[0] * (percentage/100))]

def get_info(df):
    global colname
    dic = {}
    
    dic['size'] = len(df)
    
    if dic['size'] > 0:
        dic['id'] = df[colname].values[0]
        df['SentiStrength'] = df['Positive'] - df['Negative']
        
        humans = df[df['predicted'] == 0]
        bots = df[df['predicted'] == 1]
        dic['bot_percentage'] = len(bots)/dic['size']

        dic['humans_count'] = len(humans)
        dic['bots_count'] = len(bots)

        dic['influence_mean'] = df['influence'].mean()
        dic['bots_influence_mean'] = bots['influence'].mean()
        dic['humans_influence_mean'] = humans['influence'].mean()

        dic['influence_std'] = df['influence'].std()
        dic['bots_influence_std'] = bots['influence'].std()
        dic['humans_influence_std'] = humans['influence'].std()
        
        
        percentages = [5, 10, 25]
        
        for percentage in percentages:
            t_df = get_percentage_of_post(df, percentage)
            humans = t_df[t_df['predicted'] == 0]
            bots = t_df[t_df['predicted'] == 1]
            dic['bot_percentage_{}'.format(percentage)] = len(bots)/len(t_df)

            dic['humans_count_{}'.format(percentage)] = len(humans)
            dic['bots_count_{}'.format(percentage)] = len(bots)

            dic['influence_mean_{}'.format(percentage)] = t_df['influence'].mean()
            dic['bots_influence_mean_{}'.format(percentage)] = bots['influence'].mean()
            dic['humans_influence_mean_{}'.format(percentage)] = humans['influence'].mean()

            dic['influence_std_{}'.format(percentage)] = t_df['influence'].std()
            dic['bots_influence_std_{}'.format(percentage)] = bots['influence'].std()
            dic['humans_influence_std_{}'.format(percentage)] = humans['influence'].std()
        

#         dic['vader_mean'] = df['vader_emotion'].mean()
#         dic['bots_vader_mean'] = bots['vader_emotion'].mean()
#         dic['humans_vader_mean'] = humans['vader_emotion'].mean()

#         dic['vader_std'] = df['vader_emotion'].std()
#         dic['bots_vader_std'] = bots['vader_emotion'].std()
#         dic['humans_vader_std'] = humans['vader_emotion'].std()

#         dic['polarity_mean'] = df['polarity'].mean()
#         dic['bots_polarity_mean'] = bots['polarity'].mean()
#         dic['humans_polarity_mean'] = humans['polarity'].mean()

#         dic['polarity_std'] = df['polarity'].std()
#         dic['bots_polarity_std'] = bots['polarity'].std()
#         dic['humans_polarity_std'] = humans['polarity'].std()

#         dic['subjectivity_mean'] = df['subjectivity'].mean()
#         dic['bots_subjectivity_mean'] = bots['subjectivity'].mean()
#         dic['humans_subjectivity_mean'] = humans['subjectivity'].mean()

#         dic['subjectivity_std'] = df['subjectivity'].std()
#         dic['bots_subjectivity_std'] = bots['subjectivity'].std()
#         dic['humans_subjectivity_std'] = humans['subjectivity'].std()

        dic['SentiStrength_mean'] = df['SentiStrength'].mean()
        dic['bots_SentiStrength_mean'] = bots['SentiStrength'].mean()
        dic['humans_SentiStrength_mean'] = humans['SentiStrength'].mean()

        dic['SentiStrength_std'] = df['SentiStrength'].std()
        dic['bots_SentiStrength_std'] = bots['SentiStrength'].std()
        dic['humans_SentiStrength_std'] = humans['SentiStrength'].std()
        
    return dic

In [9]:
res = df.groupby(colname).apply(get_info)

In [10]:
n_res = pd.DataFrame.from_records(res.reset_index(drop=True).values)

In [11]:
n_res['ID'] = res.index

In [12]:
fname = "retweet_cascade"

In [13]:
n_res[['bot_percentage', 'bot_percentage_10', 'bot_percentage_25', 'bot_percentage_5']] = n_res[['bot_percentage', 'bot_percentage_10', 'bot_percentage_25', 'bot_percentage_5']] * 100

In [14]:
n_res.to_csv('metrics_{}.csv'.format(fname), index=None)